### Hierarchical Organization of Patent Data

#### Project Summary
While search engines offer a convenient way to search patents, patent data stored in large CSV files often occupy a significant amount of disk space and contain a significant amount of text, oftentimes metadata that seems esoteric to novice users and cannot be easily incorporated into machine learning models. One such example is using the International Patent Classification (IPC) alphabetical nomenclature; while this alphanumeric system effectively categorizes the field of a patent, it would be more helpful to explicitly specify the field and to partition the data based on such field(s), e.g., chemistry and physics patents. 

The following project is a data pipeline that aims to facilitate patent data organization and storage. Specifically, a Kaggle patent dataset is stored in a public S3 bucket; using AWS IAM credentials, combined with PySpark and pandas modules, the data pipeline normalizes the Kaggle patent dataset (see below), maps the IPC classification using a manually annotated IPC classification definition set (see below) and organizes the patent entries into parquet files stored on an S3 bucket. The final product is a paritioned dataset that can be seamlessly incorporated with big data frameworks such as Hadoop. While beyond the scope of this project, this reorganized and partitioned dataset could greatly facilitate machine learning and natural language processing applications.

In [1]:
#This project requires that the pandas version be at least 0.25.0
import sys
!{sys.executable} -m pip install s3fs
!{sys.executable} -m pip install pandas==0.25.0
import pandas as pd
import configparser
import s3fs 
import os
from  pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth

### AWS Credentials
Since the Kaggle dataset is stored on a public S3 bucket, there needs to be an IAM Role that permits S3 bucket read access. The access key and secret key are to be entered into the dl.cfg file without any quotation marks.

In [2]:
config=configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS_CREDS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS_CREDS']['AWS_SECRET_ACCESS_KEY']

### Data Sources
#### Patent Data Set

The first file was obtained from Kaggle in the following repository: https://www.kaggle.com/mystery/patents. 

This .csv file contains the following fields: 
- Index: An arbitrary running integer assigned based on the row number.
- Application ID: The ID given to an application, which will be assigned by the respective patent & trademark office. This is often used to distinguish international/domestic/patent cooperation treaty (PCT) patents.
- Application Number: As with the application id, the application number is a unique identifier for the patent.
- Country: The origin of the patent. Patents with the "WO" designation signify a PCT application, meaning that is an application valid for multiple countries. 
- Title: The title of the patent.
- Abstract: A general summary of the patent.
- International Patent Classification (IPC): An alphanumeric system designed to divide the patent's field into specific categories. While there is a high degree of granularity in this system, the project will only use the most general field.
- Application Date: The year in which the patent was first filed for priority date.
- Year: The year obtained from the application field
- Cluster_tf_idf: An extraneous field that was inherent to the repository.

#### IPC Nomenclature
The second file creates a mapping system that designates the first letter of the IPC into a general category for the patent's field. The following is an overview of the nomeclature:

- A: Human Necessities
- B: Performing Operations, Transporting
- C: Chemistry, Metallurgy
- D: Textiles, Paper
- E: Fixed Constructions
- F: Mechanical Engineering
- G: Physics
- H: Electricity

These categories can be further sub-categorized based on the numbers that follow the first letter. However, since these numeric values are not annotated into a JSON or CSV file, increasing the granularity of the patent's metadata is beyond the scope of this project. Further information can be found in the link below:

https://www.wipo.int/classifications/ipc/ipcpub/?notion=scheme&version=20200101&symbol=none&menulang=en&lang=en&viewmode=f&fipcpc=no&showdeleted=yes&indexes=no&headings=yes&notes=yes&direction=o2n&initial=A&cwid=none&tree=no&searchmode=smart


#### Read patent dataset from a public S3 bucket

In [3]:
#Read data from CSV file into pandas dataframe and remove the index and cluster_tf_idf fields
patent_df=pd.read_csv('s3://kaggle-patent-data/Patent_listing.csv')
patent_df=patent_df.drop('cluster_tf_idf',1)
patent_df.columns=['Index','Application_Id','Application_Number','Country','Title','Abstract','IPC','Application_Date','Year']
patent_df=patent_df.drop('Index',1)

#### Read IPC annotation dataset from a public S3 bucket

In [4]:
#Read data from CSV file into pandas dataframe
ipc_df=pd.read_csv('s3://kaggle-patent-data/ipc_classification.csv')

#### Data Quality Check
The data quality check entails finding potential duplicate values. The unique identifier for the patent is the application id, so this field is used to assess whether or not there is a duplicate entry. The second check is looking for any null/NA values. While the file has been pre-processed, quality control is essential to determine what to do with any such fields.

In [5]:
#Inspect patent dataframe for duplicated data
patent_df.duplicated(subset='Application_Id').sum()

0

In [6]:
#Inspect patent dataframe for missing values
patent_df.isnull().sum()

Application_Id         0
Application_Number     2
Country                0
Title                 18
Abstract              23
IPC                    0
Application_Date       0
Year                   0
dtype: int64

#### Data Wrangling
##### Null Values
As shown above, there were null values contained in the dataframe. Since there is only a small percentage of rows with null fields and these fields are essential to the description of the patent, they will not be included in the pipeline.

In [7]:
patent_df=patent_df.dropna()
patent_df.isnull().sum()

Application_Id        0
Application_Number    0
Country               0
Title                 0
Abstract              0
IPC                   0
Application_Date      0
Year                  0
dtype: int64

In [9]:
patent_df.head()

,Application_Id,Application_Number,Country,Title,Abstract,IPC,Application_Date,Year
0,WO2015116015,PCT/UA2014/000017,WO,INFLATABLE AIRCRAFT,inflatable vertical takeoff landing aircraft i...,B64C 29/00; B64C 31/06; B64C 27/32,2014-03-02,2014
1,WO2015166113,PCT/ES2014/070372,WO,SEALING DEVICE FOR AIRCRAFT PROPELLER ENGINE,invention relates sealing device aircraft engi...,B64C 35/00; B64C 11/00; B64C 99/00,2014-04-29,2014
2,WO2014185492,PCT/JP2014/062955,WO,VERTICAL TAKE-OFF AND LANDING AIRCRAFT,vertical take landing aircraft comprises prope...,B64C 29/00; B64C 27/20; B64C 27/22,2014-05-15,2014
3,WO2015112039,PCT/RU2014/000035,WO,AIRCRAFT,﻿ claimed invention relates aviation particula...,B64C 39/12; B64C 11/00; B64C 9/00,2014-01-22,2014
4,WO2015099375,PCT/KR2014/012634,WO,MULTI-ROTOR FLYING OBJECT,present invention provides multi rotor flying ...,B64C 27/08; B64C 29/02; B64C 27/52,2014-12-22,2014


##### Multiple IPC Categories
While the IPC classification system does offer a good general overview of patent field, there could be instances where a patent can encompass multiple categories. As shown in the dataset above, there are Application Id's that include multiple IPC designations in the "IPC" column. While this project only focuses on the first letter and does not account for increased granularity with each subcategory, the data pipeline will assume that the patent could encompass multiple general fields. Thus, the table will be denormalized by exploding the values of the IPC categories into mutliple rows. 

Based on the structure of the dataset, the data may be split using "; " as the delimiter. However, to accomodate for any potential errors in the file structure, the values will be looped through until reaching a letter that falls between "A" and "H". Through this method, the IPC categories will be mapped to the respect field. For example, an entry such as "B64C 29/00" will be mapped to "Performing Operations."

In [10]:
patent_df['IPC'] = patent_df['IPC'].str.split('; ')
patent_df_flattened=patent_df.explode('IPC')

In [11]:
#Generate an IPC listing and a category listing which will map each IPC category
category_listing=[]
ipc_list=list(patent_df_flattened.IPC)
ipc_list_space_modified = []

#Remove any potential leading or trailing whitespaces between each IPC entry
for elements in ipc_list:
    ipc_list_space_modified.append(elements.strip())

In [12]:
#Conver the IPC dataframe into a dictionary that can be used for mapping
#Generate a category listing that will then be added into the patents dataframe

category_dict=ipc_df.set_index('Section').T.to_dict('list')
categories=set(category_dict.keys())

for element in ipc_list_space_modified:
    try:
        category_listing.append((category_dict[element[0]][0]))
        
    except:
        for i in range(0,len(element)):
            if element[i] in categories:
                category_listing.append(element[i])
                break
            else:
                continue

In [13]:
patent_df_flattened['IPC_Description']=pd.Series(category_listing)

In [14]:
patent_df_flattened.head()

,Application_Id,Application_Number,Country,Title,Abstract,IPC,Application_Date,Year,IPC_Description
0,WO2015116015,PCT/UA2014/000017,WO,INFLATABLE AIRCRAFT,inflatable vertical takeoff landing aircraft i...,B64C 29/00,2014-03-02,2014,Performing Operations
0,WO2015116015,PCT/UA2014/000017,WO,INFLATABLE AIRCRAFT,inflatable vertical takeoff landing aircraft i...,B64C 31/06,2014-03-02,2014,Performing Operations
0,WO2015116015,PCT/UA2014/000017,WO,INFLATABLE AIRCRAFT,inflatable vertical takeoff landing aircraft i...,B64C 27/32,2014-03-02,2014,Performing Operations
1,WO2015166113,PCT/ES2014/070372,WO,SEALING DEVICE FOR AIRCRAFT PROPELLER ENGINE,invention relates sealing device aircraft engi...,B64C 35/00,2014-04-29,2014,Performing Operations
1,WO2015166113,PCT/ES2014/070372,WO,SEALING DEVICE FOR AIRCRAFT PROPELLER ENGINE,invention relates sealing device aircraft engi...,B64C 11/00,2014-04-29,2014,Performing Operations


#### Data Model
Since there are potentially multiple IPC categories for each patent, the dataframe has undergone some degree of denormalization. The schema for the patents datasets entails partitioning the data based on specific fields, i.e., IPC category and country, making a Pyspark dataframe ideal. The dataframe will be converted into a SQL table, which will then be written into parquet datafiles stored in a public S3 bucket of choice.

In [15]:
#Convert all dataframe contents into string to ensure proper conversion of pandas dataframe into Pyspark dataframe
patent_df_flattened= patent_df_flattened.applymap(str)

In [16]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [17]:
spark=create_spark_session()

spark_patent_df=spark.createDataFrame(patent_df_flattened)

In [18]:
#Create Pyspark data table
spark_patent_df.createOrReplaceTempView("patents")

In [19]:
patent_data = spark.sql("""SELECT  DISTINCT Application_Id as Application_Id,
                                            Application_Number as Application_Number,
                                            Country as Country,
                                            Title as Title,
                                            Abstract as Abstract, 
                                            IPC_Description as IPC_Description,
                                            Year as year,
                                            month(Application_Date) as month,
                                            dayofmonth(Application_Date) as day
                           
                           FROM patents""")

In [24]:
def write_parquet(pyspark_df, s3_path):
    try:
        pyspark_df.write.mode("overwrite").partitionBy("IPC_Description","Country").parquet(s3_path)
    except:
        print("There is an error writing parquet file to specific S3 bucket")

In [20]:
s3_bucket_path = ".parquet"

write_parquet(patent_data, s3_bucket_path)

#### Closing Remarks
Patents data is very extensive, and even a relatively small file can be quite cumbersome to organize/query on a relational database. Through this methodology, we create a scalable-friendly storage system that can facilitate data organization and retrieval with big data solutions such as Hadoop. Should this dataset become extensive and receive continual data, this project can easily be incorporated into an Apache Airflow DAG, where we can automate the intervals at which the pipeline operates. 